# Otsu's Thresholding Demo
此 Notebook 示範如何使用 Otsu 閾值分割法進行影像二值化，並透過 Gradio 建立互動式介面。

## 匯入所需的函式庫與定義核心函式
以下程式碼匯入必要的函式庫，並定義計算 Otsu 閾值與 Gradio 呼叫函式。

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr

def compute_otsu_threshold(image: np.ndarray) -> int:
    """手動計算 Otsu 閾值"""
    hist, _ = np.histogram(image.ravel(), bins=256, range=(0, 256))
    total = image.size
    cumulative_sum   = np.cumsum(hist)
    cumulative_mean  = np.cumsum(hist * np.arange(256))
    global_mean      = cumulative_mean[-1] / total
    sigma_b_squared  = (global_mean * cumulative_sum - cumulative_mean) ** 2 \
                       / (cumulative_sum * (total - cumulative_sum) + 1e-8)
    return int(np.argmax(sigma_b_squared))

def otsu_gradio(image: np.ndarray):
    """
    Gradio 呼叫函式：  
    1. 將輸入轉灰階  
    2. 計算 Otsu 閾值並二值化  
    3. 繪製直方圖  
    回傳：(灰階圖, 直方圖 Figure, 二值圖)
    """
    # 1. 轉為灰階
    if image.ndim == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    # 2. Otsu 閾值分割
    th = compute_otsu_threshold(gray)
    _, binary = cv2.threshold(gray, th, 255, cv2.THRESH_BINARY)
    # 3. 繪製直方圖
    fig = plt.figure(figsize=(4,3))
    plt.hist(gray.ravel(), bins=256, range=(0,255), color='gray')
    plt.axvline(th, color='red', linestyle='--', label=f"Otsu = {th}")
    plt.legend()
    plt.title("Histogram")
    plt.xlabel("Gray Level")
    plt.ylabel("Frequency")
    plt.tight_layout()
    # Gradio 接受 numpy array & matplotlib Figure
    return gray, fig, binary

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## 建立 Gradio 介面
以下程式碼建立 Gradio 介面，允許使用者上傳影像並即時顯示灰階圖、直方圖與二值化結果。

In [ ]:
iface = gr.Interface(
    fn=otsu_gradio,
    inputs=gr.Image(type="numpy", label="Upload Image"),
    outputs=[
        gr.Image(type="numpy", label="Grayscale Image"),
        gr.Plot(label="Histogram"),
        gr.Image(type="numpy", label="Binary Image")
    ],
    title="Otsu's Thresholding Demo",
    description="上傳一張影像，自動計算並顯示灰階圖、直方圖與二值化結果。"
)

## 啟動介面
執行以下程式碼以啟動 Gradio 介面。

In [ ]:
iface.launch(share=False)